# Question: can we run the lagrange filter on the already modally decomposed outputs? 

This would save running 100 z levels! Run on 8 modes or so instead

In [29]:
import sys
sys.path.append("/home/149/ab8992/tasman-tides/")
import xarray as xr
import ttidelib as tt
import scipy
import cmocean
import os
from pathlib import Path
cmap = cmocean.cm.dense_r
import matplotlib
import matplotlib.pyplot as plt
earth_cmap = matplotlib.colormaps["gist_earth"]
from datetime import timedelta

# import filtering
import numpy as np
import dask
dask.config.set({'logging.distributed': 'error'})
from dask.distributed import Client,default_client
import xrft


client = tt.startdask()
client

<Client: 'tcp://127.0.0.1:34973' processes=0 threads=0, memory=0 B>


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34973,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: 13 minutes ago,Total memory: 0 B


Process Dask Worker process (from Nanny):
2024-12-19 14:46:09,326 - distributed.nanny - ERROR - Worker process died unexpectedly
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/process.py", line 202, in _run
    target(*args, **kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/nanny.py", line 1019, in _run
    asyncio_run(run(), loop_factory=get_loop_factory())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/compatibility.py", line 236, in asyncio_run
    return loo

## Regular way: Filter THEN decompose:


In [9]:
def get_bandpass(expt,t0):
    highpass = tt.collect_data(
                exptname=expt,
                lfiltered = f"{t0}-highpass"
            ).sel(yb = slice(-120,120)).chunk({"xb":-1,"time":-1,"xb" : 50,"yb" : 50})
    lowpass = tt.collect_data(
                exptname=expt,
                lfiltered = f"{t0}-lowpass"
            ).sel(yb = slice(-120,120)).chunk({"xb":-1,"time":-1,"xb" : 50,"yb" : 50})

    u = lowpass.u - highpass.u
    v = lowpass.v - highpass.v
    cst = lowpass.cst - highpass.cst
    out = xr.merge([u.rename("u"),v.rename("v"),cst.rename("cst"),lowpass.bathy])
    out.assign_coords(
        {"time":lowpass.time * 3600,
         "xb":lowpass.xb * 1000,
         "yb":lowpass.yb * 1000}
    )
    return out

full20 = get_bandpass("full-20","40000")
efunc = xr.open_dataset("/g/data/nm03/ab8992/postprocessed/full-20/VerticalEigenfunctions-40000.nc")

U = (full20.u * efunc).integrate("zl")
V = (full20.v * efunc).integrate("zl")


Timerange as inferred from lfiltered data: (39950.0, 40049.0)
Timerange as inferred from lfiltered data: (39950.0, 40049.0)


## Try vmode decomp THEN lfilter

In [28]:
## Decompose raw fields into modes
raw_data = tt.collect_data(
                exptname="full-20",
                rawdata= ["u","v"],
                timerange=[40000 - 200,40000 + 200]
            )



u_decomposed = (raw_data.u * efunc.U).fillna(0).integrate("zl")
v_decomposed = (raw_data.v * efunc.U).fillna(0).integrate("zl")

u_decomposed = u_decomposed.assign_coords(
    {"time":u_decomposed.time * 3600,
     "xb":u_decomposed.xb * 1000,
     "yb":u_decomposed.yb * 1000}
)
v_decomposed = v_decomposed.assign_coords(
    {"time":u_decomposed.time * 3600,
     "xb":u_decomposed.xb * 1000,
     "yb":u_decomposed.yb * 1000}
)
for mode in range(5):
    u_decomposed.sel(mode = mode).to_netcdf(f"/g/data/nm03/ab8992/postprocessed/full-20/decomposed/u_mode{mode}.nc")
    v_decomposed.sel(mode = mode).to_netcdf(f"/g/data/nm03/ab8992/postprocessed/full-20/decomposed/v_mode{mode}.nc")


loading u...	

Exception ignored in: <function CachingFileManager.__del__ at 0x149873dbc0d0>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xarray/backends/file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xarray/backends/file_manager.py", line 234, in close
    file.close()
  File "src/netCDF4/_netCDF4.pyx", line 2627, in netCDF4._netCDF4.Dataset.close
  File "src/netCDF4/_netCDF4.pyx", line 2590, in netCDF4._netCDF4.Dataset._close
  File "src/netCDF4/_netCDF4.pyx", line 2034, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: Not a valid ID


done.
loading v...	done.


/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py:3357: UserWarning: Sending large graph of size 140.05 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2024-12-19 14:41:27,670 - distributed.worker.memory - WARNING - Worker is at 84% memory usage. Pausing worker.  Process memory: 13.51 GiB -- Worker memory limit: 16.00 GiB
2024-12-19 14:41:27,932 - distributed.worker.memory - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 13.35 GiB -- Worker memory limit: 16.00 GiB
2024-12-19 14:41:48,507 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:40077
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/g/data/hh5/public/apps/miniconda3/envs/

KeyboardInterrupt: 

In [27]:
u_decomposed.isel(mode = 0,time = 50).plot()

ValueError: Dataset.plot cannot be called directly. Use an explicit plot method, e.g. ds.plot.scatter(...)

In [ ]:
def PlotVmodes(data):
    fig,ax = plt.subplots(5,1,figsize = (30,18))

    for i in range(len(data.mode)):

        tp = data.U.isel(mode = i).sel(xb = slice(0,1400))
        vm = 0.7 * np.max(np.abs(tp))
        tp.plot(ax = ax[i],vmin = -1 * vm,vmax = vm,cmap = "RdBu_r")

        tt.plot_topo(ax[i],bathy = data.bathy)
        tt.plot_topo(ax[i],bathy = data.bathy)
        ax[i].set_xlim(0,1400)
        ax[i].set_xlim(0,1400)
        ax[i].set_title("")
        ax[i].set_title("")
        ax[i].set_xlabel("")
        ax[i].set_ylabel("")
        ax[i].set_yticks([])
        ax[i].set_xlabel("")
        ax[i].set_ylabel("Distance to beam centre (km)")
        ax[i].set_aspect('auto')
        ax[i].set_aspect('auto')

    ax[-1].set_xlabel("Distance from Tas (km)")
    ax[-1].set_xlabel("Distance from Tas (km)")
    fig.subplots_adjust(wspace=0.01)
    ax[0].set_title("u mode 1")
    # ax[0].set_title("v mode 1")
    return fig

PlotVmodes(xr.merge([U.rename("U"),V.rename("V"),full20.bathy.rename("bathy")]).isel(time = 50))

In [32]:
def lagrange_filter(time_window = 149,filter_cutoff = 2*np.pi/(16*3600),buffer = 48):
    """
    time window: total time that'll be filtered and appear in outputs
    buffer: the advection time for the particles. 48hrs forward and back
    """
    print("START LAGRANGE FILTERING")
    print("import filtering package:")
    import filtering

    """
    Apply the Lagrange filter to the input data. This is a wrapper around the LagrangeFilter class in filtering.py
    Saves the outputs to `postprocessing/expt/lfiltered/t0-<t0>/filtered_<zl>.nc` with a separate file for each z level.
    Inputs:
        expt: str, the experiment to process
        zl: int, the z level(s) to process
        t0: int, the middle time of the time slice we care about
    """
    HighFilterCutoff = 2*np.pi/(9.8*3600)  ## Same as tolerences for bandpass in Waterhouse 2018
    LowFilterCutoff = 2*np.pi/(15*3600)
    time_window = 5

    client = tt.startdask()
    dask.config.set(scheduler="single-threaded")
    tmpstorage = os.getenv('PBS_JOBFS')

    u = xr.open_dataset("/g/data/nm03/ab8992/postprocessed/full-20/decomposed/u_mode0.nc")
    v = xr.open_dataset("/g/data/nm03/ab8992/postprocessed/full-20/decomposed/v_mode0.nc")
    rawdata = xr.merge([u.__xarray_dataarray_variable__.rename("u"),v.__xarray_dataarray_variable__.rename("v")])
    # Save attributes to re-add later
    attrs = {
        "u": u.attrs,
        "v": v.attrs,
        "time": u.time.attrs,
        "xb": u.xb.attrs,
        "yb": u.yb.attrs,
        "base" : u.attrs
    }



    # Strip attrs since lagrange filter complains about them
    rawdata.u.attrs = {}
    rawdata.v.attrs = {}
    rawdata.time.attrs = {}
    rawdata.yb.attrs = {}
    rawdata.xb.attrs = {}
    # rawdata = rawdata.assign_coords({
    #     "time":rawdata.time * 3600,
    #     "xb":rawdata.xb * 1000,
    #     "yb":rawdata.yb * 1000})
    rawdata = rawdata.drop_vars(["lat","lon"]) 

    print("Saving data to temporary storage")
    rawdata.u.to_netcdf(tmpstorage + f"/u.nc",mode="w")
    rawdata.v.to_netcdf(tmpstorage + f"/v.nc",mode="w")
    print("done")
    client.close() ## Have to close dask client or it messes up the filtering package
    if os.path.exists(tmpstorage + "/highpass.nc"):
        os.remove(tmpstorage + "/highpass.nc")
    f_low = filtering.LagrangeFilter(
        tmpstorage + "/highpass", ## Save intermediate output to temporary storage
        {
            "U":tmpstorage + "/u.nc",
            "V":tmpstorage + "/v.nc",
        }, 
        {"U":"u","V":"v"}, 
        {"lon":"xb","lat":"yb","time":"time"},
        sample_variables=["U","V"], mesh="flat",highpass_frequency = LowFilterCutoff,
        advection_dt =timedelta(minutes=5).total_seconds(),
        window_size = timedelta(hours=buffer).total_seconds()
    )
    print("lowpass filter")
    ## Ensure we take times either side of the point of interest
    f_low(times = range(3600 * buffer,3600 * (time_window + buffer),3600)) 
    

    ## Remove stored data to save space on disk
    for i in ["u","v"]:
        os.remove(tmpstorage + f"/{i}.nc")

    client = tt.startdask()
    print(os.listdir(tmpstorage))
    ## Load the filtered data
    lowpass = xr.open_dataset(tmpstorage + "/lowpass.nc",chunks = "auto")
    lowpass.to_netcdf("/g/data/nm03/ab8992/postprocessed/full-20/decomposed/filtered_mode0")

lagrange_filter()

START LAGRANGE FILTERING
import filtering package:
<Client: 'tcp://127.0.0.1:34973' processes=1 threads=4, memory=16.00 GiB>
Saving data to temporary storage


/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/dask/base.py:1487: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


done


sh: None: command not found
INFO: Compiled SamplingParticlesample_kernel ==> /jobfs/131065674.gadi-pbs/parcels-16078/33a9a8470c1c00a7986bb0eee310c8ef_0.so
INFO:parcels.tools.loggers:Compiled SamplingParticlesample_kernel ==> /jobfs/131065674.gadi-pbs/parcels-16078/33a9a8470c1c00a7986bb0eee310c8ef_0.so
sh: None: command not found
INFO: Compiled SamplingParticleAdvectionRK4sample_kernel ==> /jobfs/131065674.gadi-pbs/parcels-16078/eeff492a2fe6a4ecb051851482bf43a0_0.so
INFO:parcels.tools.loggers:Compiled SamplingParticleAdvectionRK4sample_kernel ==> /jobfs/131065674.gadi-pbs/parcels-16078/eeff492a2fe6a4ecb051851482bf43a0_0.so
/home/149/ab8992/.local/lib/python3.10/site-packages/parcels/field.py:1325: RuntimeWarning: invalid value encountered in cast
  data = lib.concatenate([data_to_concat, data[tindex+1:, :]], axis=0)


lowpass filter


/home/149/ab8992/.local/lib/python3.10/site-packages/parcels/field.py:1327: RuntimeWarning: invalid value encountered in cast
  data = lib.concatenate([data[:tindex, :], data_to_concat], axis=0)
INFO: Temporary output files are stored in /home/149/ab8992/tmpssxz9z1h.h5.
INFO:parcels.tools.loggers:Temporary output files are stored in /home/149/ab8992/tmpssxz9z1h.h5.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/149/ab8992/tmpssxz9z1h.h5" to convert these to a NetCDF file during the run.
INFO:parcels.tools.loggers:You can use "parcels_convert_npydir_to_netcdf /home/149/ab8992/tmpssxz9z1h.h5" to convert these to a NetCDF file during the run.
100% (172800.0 of 172800.0) |############| Elapsed Time: 0:00:13 Time:  0:00:130001
INFO: Temporary output files are stored in /home/149/ab8992/tmpssxz9z1h.h5.
INFO:parcels.tools.loggers:Temporary output files are stored in /home/149/ab8992/tmpssxz9z1h.h5.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/149/ab8992/tmpssxz9z1h.h5"

<Client: 'tcp://127.0.0.1:45999' processes=7 threads=28, memory=112.00 GiB>
['dask-scratch-space', 'ompi.gadi-cpu-clx-0645.16078', 'parcels-16078', 'vv.nc', 'uu.nc', 'uv.nc', 'highpass.nc']


FileNotFoundError: [Errno 2] No such file or directory: '/jobfs/131065674.gadi-pbs/lowpass.nc'

In [34]:
tmpstorage = os.getenv('PBS_JOBFS')

out = xr.open_dataset(tmpstorage + "/highpass.nc",chunks = "auto")

In [49]:
os.listdir(tmpstorage)

['dask-scratch-space',
 'ompi.gadi-cpu-clx-0645.16078',
 'parcels-16078',
 'vv.nc',
 'uu.nc',
 'uv.nc',
 'highpass.nc']

In [7]:
u = xr.open_dataset("/g/data/nm03/ab8992/postprocessed/full-20/decomposed/u_mode0.nc")
u.time.values / 1e8

array([1.4328  , 1.432836, 1.432872, 1.432908, 1.432944, 1.43298 ,
       1.433016, 1.433052, 1.433088, 1.433124, 1.43316 , 1.433196,
       1.433232, 1.433268, 1.433304, 1.43334 , 1.433376, 1.433412,
       1.433448, 1.433484, 1.43352 , 1.433556, 1.433592, 1.433628,
       1.433664, 1.4337  , 1.433736, 1.433772, 1.433808, 1.433844,
       1.43388 , 1.433916, 1.433952, 1.433988, 1.434024, 1.43406 ,
       1.434096, 1.434132, 1.434168, 1.434204, 1.43424 , 1.434276,
       1.434312, 1.434348, 1.434384, 1.43442 , 1.434456, 1.434492,
       1.434528, 1.434564, 1.4346  , 1.434636, 1.434672, 1.434708,
       1.434744, 1.43478 , 1.434816, 1.434852, 1.434888, 1.434924,
       1.43496 , 1.434996, 1.435032, 1.435068, 1.435104, 1.43514 ,
       1.435176, 1.435212, 1.435248, 1.435284, 1.43532 , 1.435356,
       1.435392, 1.435428, 1.435464, 1.4355  , 1.435536, 1.435572,
       1.435608, 1.435644, 1.43568 , 1.435716, 1.435752, 1.435788,
       1.435824, 1.43586 , 1.435896, 1.435932, 1.435968, 1.436

In [44]:
u_decomposed.xb

<xarray.DataArray 'xb' (xb: 376)> Size: 3kB
array([     -0.,    4000.,    8000., ..., 1492000., 1496000., 1500000.])
Coordinates:
  * xb       (xb) float64 3kB -0.0 4e+03 8e+03 ... 1.492e+06 1.496e+06 1.5e+06

In [9]:
xr.open_dataset("/g/data/nm03/ab8992/postprocessed/full-20/decomposed/u_mode0.nc")

<xarray.Dataset> Size: 10kB
Dimensions:     (time: 401, yb: 1, xb: 1)
Coordinates:
    mode        int64 8B ...
  * xb          (xb) float64 8B -0.0
  * yb          (yb) float64 8B 0.0
  * time        (time) float64 3kB 1.433e+08 1.433e+08 ... 1.447e+08 1.447e+08
    lon         (yb, xb) float64 8B ...
    lat         (yb, xb) float64 8B ...
Data variables:
    Wavelength  (time, yb, xb) float64 3kB ...
    U           (time, yb, xb) float64 3kB ...